# Attendance


In [296]:
import numpy as np
import pandas as pd
from datetime import datetime, date, time, timedelta

## Python date time primer


In [297]:
# Create datetime
dt = datetime(year=2024, month=2, day=1, hour=8, minute=52)
print(dt)

2024-02-01 08:52:00


In [298]:
# Parse datetime from string
dt = datetime.strptime("2024-02-01", "%Y-%m-%d")
print(dt)

2024-02-01 00:00:00


In [299]:
# Format string
dt = datetime(year=2024, month=2, day=1, hour=8, minute=52)
dt.strftime("%Y/%m/%d %H:%M")

'2024/02/01 08:52'

In [300]:
# Create date
dt_date = date(year=2024, month=2, day=1)
print(dt_date)

2024-02-01


In [301]:
# Create time
dt_time = time(hour=7, minute=0)
print(dt_time)

07:00:00


In [302]:
# Time delta
td = timedelta(days=1)
print(td)
print(td.total_seconds())

1 day, 0:00:00
86400.0


In [303]:
dt = datetime(2024, 2, 1) + timedelta(days=1)
print(dt)

2024-02-02 00:00:00


In [304]:
# Subtract time
# https://stackoverflow.com/a/49528739
enter = time(hour=1)  # Example enter time
exit = time(hour=2)  # Example start time
enter_delta = timedelta(hours=enter.hour, minutes=enter.minute, seconds=enter.second)
exit_delta = timedelta(hours=exit.hour, minutes=exit.minute, seconds=exit.second)
difference_delta = exit_delta - enter_delta
print(difference_delta.total_seconds() / 60)

60.0


## Attendance Analysis


In [305]:
dfr = pd.read_excel("./data/clock_inout.xlsx")

In [306]:
dfr.head()

,รหัสที่เครื่อง,รหัสพนักงาน,ชื่อ-นามสกุล,แผนก,Date,1,2,3
0,1,NaN,รุ้ง,[Start Department],1/2/2567,8:52,18:18,NaN
1,1,NaN,รุ้ง,[Start Department],2/2/2567,9:00,18:14,NaN
2,1,NaN,รุ้ง,[Start Department],3/2/2567,9:23,17:03,NaN
3,1,NaN,รุ้ง,[Start Department],4/2/2567,NaN,NaN,NaN
4,1,NaN,รุ้ง,[Start Department],5/2/2567,8:46,17:26,NaN


In [307]:
dfr = dfr[["ชื่อ-นามสกุล", "Date", 1, 2, 3]].rename(
    columns={"ชื่อ-นามสกุล": "name", "Date": "date", 1: "c1", 2: "c2", 3: "c3"}
)
display(dfr.head(3))

,name,date,c1,c2,c3
0,รุ้ง,1/2/2567,8:52,18:18,NaN
1,รุ้ง,2/2/2567,9:00,18:14,NaN
2,รุ้ง,3/2/2567,9:23,17:03,NaN


In [308]:
dfr.duplicated().sum()

np.int64(0)

In [309]:
print(dfr.shape)
filtNull = dfr[["c1", "c2", "c3"]].isnull().all(axis=1)
dfr = dfr[~filtNull]
dfr.shape

(464, 5)


(335, 5)

In [310]:
def parseDate(dateStr):
    sp = dateStr.split("/")
    day = sp[0]
    month = sp[1]
    year = int(sp[2]) - 543
    return pd.to_datetime(f"{year}/{month}/{day}", format="%Y/%m/%d")


# Convert to datetime
dfr["date"] = dfr["date"].apply(parseDate)

# Convert to date
dfr["date"] = dfr["date"].dt.date

# Check
print(type(dfr["date"].iloc[0]))

<class 'datetime.date'>


In [311]:
# Convert to time
dfr["c1"] = pd.to_datetime(dfr["c1"], format="%H:%M").dt.time
dfr["c2"] = pd.to_datetime(dfr["c2"], format="%H:%M").dt.time
dfr["c3"] = pd.to_datetime(dfr["c3"], format="%H:%M").dt.time


In [312]:
dfr.head()

,name,date,c1,c2,c3
0,รุ้ง,2024-02-01,08:52:00,18:18:00,NaT
1,รุ้ง,2024-02-02,09:00:00,18:14:00,NaT
2,รุ้ง,2024-02-03,09:23:00,17:03:00,NaT
4,รุ้ง,2024-02-05,08:46:00,17:26:00,NaT
5,รุ้ง,2024-02-06,08:52:00,17:56:00,NaT


In [313]:
# Remove rows with hours outside acceptable range
def checkTimeOutsideRange(sr):
    return (sr < time(hour=6)) | (sr > time(hour=22))


filt1 = checkTimeOutsideRange(dfr["c1"])
filt2 = checkTimeOutsideRange(dfr["c2"])
filt3 = checkTimeOutsideRange(dfr["c3"])

filtOutsideRange = filt1 | filt2 | filt3
dfr[filtOutsideRange]

,name,date,c1,c2,c3
39,แปง,2024-02-11,00:00:00,NaT,NaT
126,5,2024-02-11,00:35:00,NaT,NaT
329,เมือง,2024-02-11,00:00:00,NaT,NaT
416,ไอซ์,2024-02-11,00:39:00,NaT,NaT


In [314]:
dfr = dfr[~filtOutsideRange]

In [315]:
dfr["incompleteInOut"] = False
filtOneCheckIn = (~dfr[["c1", "c2", "c3"]].isnull()).sum(axis=1) == 1
dfr.loc[filtOneCheckIn, "incompleteInOut"] = True

In [316]:
# Add time for incomplete check-in/out
def addTimeForIncompleteCheckInOut(sr):
    dt_time = sr["c1"]
    # Determine whether the missing is the morning in or evening out.
    if dt_time < time(hour=13):  # 1pm
        sr["c2"] = time(hour=18)  # 6pm
    else:
        sr["c2"] = time(hour=9)  # 9am
    return sr


filtIIO = dfr["incompleteInOut"]
dfr.loc[filtIIO, :] = dfr.loc[filtIIO, :].apply(addTimeForIncompleteCheckInOut, axis=1)

In [317]:
dfr.loc[filtIIO].head()

,name,date,c1,c2,c3,incompleteInOut
9,รุ้ง,2024-02-10,09:27:00,18:00:00,NaT,True
23,รุ้ง,2024-02-24,09:41:00,18:00:00,NaT,True
28,รุ้ง,2024-02-29,08:04:00,18:00:00,NaT,True
38,แปง,2024-02-10,07:50:00,18:00:00,NaT,True
57,แปง,2024-02-29,07:49:00,18:00:00,NaT,True


In [318]:
def calculateInOut(row):
    times = row.loc[["c1", "c2", "c3"]].dropna()
    res = times.agg(["min", "max"])
    return pd.concat([row, res])


dfr = dfr.apply(calculateInOut, axis=1)
dfr = dfr.rename(columns={"min": "in", "max": "out"})
dfr.head()

,name,date,c1,c2,c3,incompleteInOut,in,out
0,รุ้ง,2024-02-01,08:52:00,18:18:00,NaT,False,08:52:00,18:18:00
1,รุ้ง,2024-02-02,09:00:00,18:14:00,NaT,False,09:00:00,18:14:00
2,รุ้ง,2024-02-03,09:23:00,17:03:00,NaT,False,09:23:00,17:03:00
4,รุ้ง,2024-02-05,08:46:00,17:26:00,NaT,False,08:46:00,17:26:00
5,รุ้ง,2024-02-06,08:52:00,17:56:00,NaT,False,08:52:00,17:56:00


In [319]:
dfr["isInLate"] = dfr["in"] > time(hour=9)
dfr["isOutEarly"] = dfr["out"] < time(hour=18)
dfr.head()

,name,date,c1,c2,c3,incompleteInOut,in,out,isInLate,isOutEarly
0,รุ้ง,2024-02-01,08:52:00,18:18:00,NaT,False,08:52:00,18:18:00,False,False
1,รุ้ง,2024-02-02,09:00:00,18:14:00,NaT,False,09:00:00,18:14:00,False,False
2,รุ้ง,2024-02-03,09:23:00,17:03:00,NaT,False,09:23:00,17:03:00,True,True
4,รุ้ง,2024-02-05,08:46:00,17:26:00,NaT,False,08:46:00,17:26:00,False,True
5,รุ้ง,2024-02-06,08:52:00,17:56:00,NaT,False,08:52:00,17:56:00,False,True


In [320]:
# You cannot substract time and time. Need to convert to timedelta first.
def calInLateMin(dt_time):
    deltaIn = timedelta(hours=dt_time.hour, minutes=dt_time.minute)
    deltaStart = timedelta(hours=9)
    lateMin = (deltaIn - deltaStart).total_seconds() / 60
    if lateMin < 0:
        lateMin = 0
    return lateMin


dfr["inLateMin"] = dfr["in"].apply(calInLateMin)
dfr.head()

,name,date,c1,c2,c3,incompleteInOut,in,out,isInLate,isOutEarly,inLateMin
0,รุ้ง,2024-02-01,08:52:00,18:18:00,NaT,False,08:52:00,18:18:00,False,False,0.0
1,รุ้ง,2024-02-02,09:00:00,18:14:00,NaT,False,09:00:00,18:14:00,False,False,0.0
2,รุ้ง,2024-02-03,09:23:00,17:03:00,NaT,False,09:23:00,17:03:00,True,True,23.0
4,รุ้ง,2024-02-05,08:46:00,17:26:00,NaT,False,08:46:00,17:26:00,False,True,0.0
5,รุ้ง,2024-02-06,08:52:00,17:56:00,NaT,False,08:52:00,17:56:00,False,True,0.0


In [321]:
def calOutEarlyMin(dt_time):
    deltaOut = timedelta(hours=dt_time.hour, minutes=dt_time.minute)
    deltaEnd = timedelta(hours=18)
    earlyMon = (deltaEnd - deltaOut).total_seconds() / 60
    if earlyMon < 0:
        earlyMon = 0
    return earlyMon


dfr["outEarlyMin"] = dfr["out"].apply(calOutEarlyMin)
dfr.head()

,name,date,c1,c2,c3,incompleteInOut,in,out,isInLate,isOutEarly,inLateMin,outEarlyMin
0,รุ้ง,2024-02-01,08:52:00,18:18:00,NaT,False,08:52:00,18:18:00,False,False,0.0,0.0
1,รุ้ง,2024-02-02,09:00:00,18:14:00,NaT,False,09:00:00,18:14:00,False,False,0.0,0.0
2,รุ้ง,2024-02-03,09:23:00,17:03:00,NaT,False,09:23:00,17:03:00,True,True,23.0,57.0
4,รุ้ง,2024-02-05,08:46:00,17:26:00,NaT,False,08:46:00,17:26:00,False,True,0.0,34.0
5,รุ้ง,2024-02-06,08:52:00,17:56:00,NaT,False,08:52:00,17:56:00,False,True,0.0,4.0


In [322]:
def calWorkingDuration(row):
    timeIn = row["in"]
    timeOut = row["out"]
    deltaIn = timedelta(hours=timeIn.hour, minutes=timeIn.minute)
    deltaOut = timedelta(hours=timeOut.hour, minutes=timeOut.minute)
    return (deltaOut - deltaIn).total_seconds() / 60


dfr["workingDuration"] = dfr.apply(calWorkingDuration, axis=1)
dfr.head()

,name,date,c1,c2,c3,incompleteInOut,in,out,isInLate,isOutEarly,inLateMin,outEarlyMin,workingDuration
0,รุ้ง,2024-02-01,08:52:00,18:18:00,NaT,False,08:52:00,18:18:00,False,False,0.0,0.0,566.0
1,รุ้ง,2024-02-02,09:00:00,18:14:00,NaT,False,09:00:00,18:14:00,False,False,0.0,0.0,554.0
2,รุ้ง,2024-02-03,09:23:00,17:03:00,NaT,False,09:23:00,17:03:00,True,True,23.0,57.0,460.0
4,รุ้ง,2024-02-05,08:46:00,17:26:00,NaT,False,08:46:00,17:26:00,False,True,0.0,34.0,520.0
5,รุ้ง,2024-02-06,08:52:00,17:56:00,NaT,False,08:52:00,17:56:00,False,True,0.0,4.0,544.0


In [323]:
dfr["overWorkMin"] = dfr["workingDuration"] - (9 * 60)
dfr.head()

,name,date,c1,c2,c3,incompleteInOut,in,out,isInLate,isOutEarly,inLateMin,outEarlyMin,workingDuration,overWorkMin
0,รุ้ง,2024-02-01,08:52:00,18:18:00,NaT,False,08:52:00,18:18:00,False,False,0.0,0.0,566.0,26.0
1,รุ้ง,2024-02-02,09:00:00,18:14:00,NaT,False,09:00:00,18:14:00,False,False,0.0,0.0,554.0,14.0
2,รุ้ง,2024-02-03,09:23:00,17:03:00,NaT,False,09:23:00,17:03:00,True,True,23.0,57.0,460.0,-80.0
4,รุ้ง,2024-02-05,08:46:00,17:26:00,NaT,False,08:46:00,17:26:00,False,True,0.0,34.0,520.0,-20.0
5,รุ้ง,2024-02-06,08:52:00,17:56:00,NaT,False,08:52:00,17:56:00,False,True,0.0,4.0,544.0,4.0


In [324]:
# Get working days
dtRangesMonth = pd.date_range(start="2024-02-01", end="2024-02-28")

dtRangesWorking = []
for dtr in dtRangesMonth:
    if dtr.weekday() <= 5:  # Monday to Saturday
        dtRangesWorking.append(dtr.date())

In [325]:
def matchWorkingDate(dft):
    dfWorking = pd.DataFrame(data={"working_date": dtRangesWorking})
    dfm = pd.merge(dfWorking, dft, left_on="working_date", right_on="date", how="left")
    dfm["present"] = dfm["date"].notnull()
    dfm["absent"] = dfm["date"].isnull()
    return dfm


dfg = dfr.groupby(by="name")
dfgm = dfg.apply(matchWorkingDate, include_groups=False)


# Testing
# dfg = dfr.groupby(by="name")
# dft = dfg.get_group("รุ้ง")
# dfWorking = pd.DataFrame(data={"working_date": dtRangesWorking})
# dfm = pd.merge(dfWorking, dft, left_on="working_date", right_on="date", how="left")
# name = dfm["name"].value_counts().index[0]
# dfm["name"] = name
# dfm

In [326]:
dfgm = dfgm.reset_index().drop(columns="level_1")

In [327]:
dfgm.columns

Index(['name', 'working_date', 'date', 'c1', 'c2', 'c3', 'incompleteInOut',
       'in', 'out', 'isInLate', 'isOutEarly', 'inLateMin', 'outEarlyMin',
       'workingDuration', 'overWorkMin', 'present', 'absent'],
      dtype='object')

In [328]:
out1 = (
    dfgm.groupby(by=["name"])
    .agg(
        {
            "present": "sum",
            "absent": "sum",
            "workingDuration": lambda s: s.mean() / 60,
            "overWorkMin": "sum",
            "incompleteInOut": "sum",
            "inLateMin": "sum",
            "outEarlyMin": "sum",
        }
    )
    .rename(
        columns={
            "workingDuration": "workingDuration (mean)",
        }
    )
)
display(out1)

,present,absent,workingDuration (mean),overWorkMin,incompleteInOut,inLateMin,outEarlyMin
name,,,,,,,
5,23,1,9.422464,583.0,2,0.0,1169.0
จายลอด,22,2,9.196212,259.0,1,0.0,1253.0
พอง,24,0,9.236111,340.0,2,0.0,1284.0
พี่นัย,22,2,9.352273,465.0,2,53.0,1022.0
พี่น้อย (ยุง),21,3,9.086508,109.0,0,0.0,1495.0
พี่หญิง,24,0,9.239583,345.0,2,0.0,1276.0
พี่หน่อ,22,2,9.180303,238.0,1,0.0,1251.0
พี่แอม,23,1,8.392029,-839.0,4,87.0,1056.0
พี่โบ,22,2,9.270455,357.0,2,0.0,1109.0


In [329]:
out2 = dfgm.pivot(index="name", columns="working_date", values="overWorkMin")
display(out2)

working_date,2024-02-01,2024-02-02,2024-02-03,2024-02-05,2024-02-06,2024-02-07,2024-02-08,2024-02-09,2024-02-10,2024-02-12,...,2024-02-17,2024-02-19,2024-02-20,2024-02-21,2024-02-22,2024-02-23,2024-02-24,2024-02-26,2024-02-27,2024-02-28
name,,,,,,,,,,,,,,,,,,,,,
5,NaN,19.0,54.0,9.0,14.0,13.0,13.0,16.0,79.0,26.0,...,16.0,16.0,24.0,33.0,27.0,15.0,64.0,17.0,23.0,31.0
จายลอด,6.0,12.0,11.0,2.0,10.0,8.0,8.0,9.0,NaN,14.0,...,15.0,9.0,8.0,NaN,10.0,2.0,72.0,12.0,3.0,12.0
พอง,7.0,7.0,11.0,8.0,11.0,8.0,14.0,15.0,65.0,7.0,...,11.0,10.0,10.0,12.0,6.0,6.0,70.0,10.0,8.0,8.0
พี่นัย,12.0,7.0,9.0,8.0,NaN,15.0,28.0,99.0,74.0,11.0,...,-109.0,NaN,10.0,11.0,11.0,13.0,80.0,15.0,50.0,15.0
พี่น้อย (ยุง),13.0,34.0,12.0,6.0,29.0,24.0,20.0,25.0,NaN,19.0,...,11.0,19.0,12.0,13.0,13.0,-241.0,NaN,NaN,31.0,5.0
พี่หญิง,7.0,7.0,10.0,8.0,11.0,9.0,7.0,15.0,65.0,7.0,...,9.0,10.0,10.0,14.0,7.0,7.0,70.0,10.0,9.0,7.0
พี่หน่อ,14.0,6.0,2.0,9.0,16.0,7.0,2.0,-3.0,74.0,14.0,...,17.0,1.0,NaN,12.0,-1.0,17.0,NaN,10.0,7.0,8.0
พี่แอม,-6.0,-16.0,-28.0,-51.0,-55.0,-56.0,7.0,-35.0,-10.0,-36.0,...,-48.0,-46.0,-129.0,-30.0,-24.0,-39.0,17.0,-40.0,-45.0,-30.0
พี่โบ,16.0,12.0,5.0,5.0,NaN,9.0,22.0,38.0,66.0,12.0,...,5.0,NaN,5.0,11.0,1.0,6.0,72.0,5.0,16.0,13.0


In [330]:
# Using pivot_table, not pivot
out3 = dfgm.pivot_table(
    index="name", columns="working_date", values="present", aggfunc="sum"
)
display(out3)

working_date,2024-02-01,2024-02-02,2024-02-03,2024-02-05,2024-02-06,2024-02-07,2024-02-08,2024-02-09,2024-02-10,2024-02-12,...,2024-02-17,2024-02-19,2024-02-20,2024-02-21,2024-02-22,2024-02-23,2024-02-24,2024-02-26,2024-02-27,2024-02-28
name,,,,,,,,,,,,,,,,,,,,,
5,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
จายลอด,1,1,1,1,1,1,1,1,0,1,...,1,1,1,0,1,1,1,1,1,1
พอง,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
พี่นัย,1,1,1,1,0,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1
พี่น้อย (ยุง),1,1,1,1,1,1,1,1,0,1,...,1,1,1,1,1,1,0,0,1,1
พี่หญิง,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
พี่หน่อ,1,1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,0,1,1,1
พี่แอม,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
พี่โบ,1,1,1,1,0,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1


In [331]:
# Using pivot_table, not pivot
out4 = dfgm.pivot_table(
    index="name",
    columns="working_date",
    values="workingDuration",
    aggfunc=lambda s: np.round(s.mean() / 60, 2),
)
display(out4)

working_date,2024-02-01,2024-02-02,2024-02-03,2024-02-05,2024-02-06,2024-02-07,2024-02-08,2024-02-09,2024-02-10,2024-02-12,...,2024-02-17,2024-02-19,2024-02-20,2024-02-21,2024-02-22,2024-02-23,2024-02-24,2024-02-26,2024-02-27,2024-02-28
name,,,,,,,,,,,,,,,,,,,,,
5,NaN,9.32,9.90,9.15,9.23,9.22,9.22,9.27,10.32,9.43,...,9.27,9.27,9.40,9.55,9.45,9.25,10.07,9.28,9.38,9.52
จายลอด,9.10,9.20,9.18,9.03,9.17,9.13,9.13,9.15,NaN,9.23,...,9.25,9.15,9.13,NaN,9.17,9.03,10.20,9.20,9.05,9.20
พอง,9.12,9.12,9.18,9.13,9.18,9.13,9.23,9.25,10.08,9.12,...,9.18,9.17,9.17,9.20,9.10,9.10,10.17,9.17,9.13,9.13
พี่นัย,9.20,9.12,9.15,9.13,NaN,9.25,9.47,10.65,10.23,9.18,...,7.18,NaN,9.17,9.18,9.18,9.22,10.33,9.25,9.83,9.25
พี่น้อย (ยุง),9.22,9.57,9.20,9.10,9.48,9.40,9.33,9.42,NaN,9.32,...,9.18,9.32,9.20,9.22,9.22,4.98,NaN,NaN,9.52,9.08
พี่หญิง,9.12,9.12,9.17,9.13,9.18,9.15,9.12,9.25,10.08,9.12,...,9.15,9.17,9.17,9.23,9.12,9.12,10.17,9.17,9.15,9.12
พี่หน่อ,9.23,9.10,9.03,9.15,9.27,9.12,9.03,8.95,10.23,9.23,...,9.28,9.02,NaN,9.20,8.98,9.28,NaN,9.17,9.12,9.13
พี่แอม,8.90,8.73,8.53,8.15,8.08,8.07,9.12,8.42,8.83,8.40,...,8.20,8.23,6.85,8.50,8.60,8.35,9.28,8.33,8.25,8.50
พี่โบ,9.27,9.20,9.08,9.08,NaN,9.15,9.37,9.63,10.10,9.20,...,9.08,NaN,9.08,9.18,9.02,9.10,10.20,9.08,9.27,9.22


In [332]:
# Write dataframes to Excel with multiple sheets
names = ["summary", "overWorkMin", "present", "workingDuration (hour)"]
dataframes = [out1, out2, out3, out4]
with pd.ExcelWriter("out_attendance.xlsx") as writer:
    for name, frame in zip(names, dataframes):
        frame.to_excel(writer, sheet_name=name, index=True)